simplified version of the vLEI ecosystem trust chain.

We present here a simplified version of the vLEI trust chain. Using the real schema definitions, but simplifying the structure of the Identifiers. The production version relies on multisig and AID delegation. These examples simplify that part. To see how the chain is constructed, see [link to wvi_workflow repo] 

## Setup Phase
Creates 4 different identity clients: GLEIF, QVI, LE, and Role
Establishes connections between all parties

Creates credential registries for issuers

In [1]:
import { randomPasscode, Saider, Serder} from 'npm:signify-ts@0.3.0-rc1';
import { initializeSignify, 
         initializeAndConnectClient,
         createNewAID,
         addEndRoleForAID,
         generateOOBI,
         resolveOOBI,
         createTimestamp,
         createCredentialRegistry,
         getSchema,
         issueCredential,
         ipexGrantCredential,
         getCredentialState,
         waitForAndGetNotification,
         ipexAdmitGrant,
         markNotificationRead,
         DEFAULT_IDENTIFIER_ARGS,
         DEFAULT_TIMEOUT_MS,
         DEFAULT_DELAY_MS,
         DEFAULT_RETRIES,
         ROLE_AGENT,
         IPEX_GRANT_ROUTE,
         IPEX_ADMIT_ROUTE,
         IPEX_APPLY_ROUTE,
         IPEX_OFFER_ROUTE,
         SCHEMA_SERVER_HOST
       } from './scripts_ts/utils.ts';

// Create clients, AIDs and OOBIs.

const gleifBran = randomPasscode()
const gleifAlias = 'gleif'
const { client: gleifClient } = await initializeAndConnectClient(gleifBran)
const { aid: gleifAid} = await createNewAID(gleifClient, gleifAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(gleifClient, gleifAlias, ROLE_AGENT);
const gleifOOBI = await generateOOBI(gleifClient, gleifAlias, ROLE_AGENT);

const qviBran = randomPasscode()
const qviAlias = 'qvi'
const { client: qviClient } = await initializeAndConnectClient(qviBran)
const { aid: qviAid} = await createNewAID(qviClient, qviAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(qviClient, qviAlias, ROLE_AGENT);
const qviOOBI = await generateOOBI(qviClient, qviAlias, ROLE_AGENT);

const leBran = randomPasscode()
const leAlias = 'le'
const { client: leClient } = await initializeAndConnectClient(leBran)
const { aid: leAid} = await createNewAID(leClient, leAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(leClient, leAlias, ROLE_AGENT);
const leOOBI = await generateOOBI(leClient, leAlias, ROLE_AGENT);

const roleBran = randomPasscode()
const roleAlias = 'role'
const { client: roleClient } = await initializeAndConnectClient(roleBran)
const { aid: roleAid} = await createNewAID(roleClient, roleAlias, DEFAULT_IDENTIFIER_ARGS);
await addEndRoleForAID(roleClient, roleAlias, ROLE_AGENT);
const roleOOBI = await generateOOBI(roleClient, roleAlias, ROLE_AGENT);


// Client OOBI resolution (Create contacts)
await Promise.all([
    resolveOOBI(gleifClient, qviOOBI, qviAlias),
    resolveOOBI(qviClient, gleifOOBI, gleifAlias),
    resolveOOBI(qviClient, leOOBI, leAlias),
    resolveOOBI(qviClient, roleOOBI, roleAlias),
    resolveOOBI(leClient, gleifOOBI, gleifAlias),
    resolveOOBI(leClient, qviOOBI, qviAlias),
    resolveOOBI(leClient, roleOOBI, roleAlias),
    resolveOOBI(roleClient, gleifOOBI, gleifAlias),
    resolveOOBI(roleClient, leOOBI, leAlias),
    resolveOOBI(roleClient, qviOOBI, qviAlias)
]);

// Create Issuer Credential Registry


const { registrySaid: gleifRegistrySaid } = await createCredentialRegistry(gleifClient, gleifAlias, 'gleifRegistry')
const { registrySaid: qviRegistrySaid } = createCredentialRegistry(qviClient, qviAlias, 'qviRegistry')
const { registrySaid: leRegistrySaid } = createCredentialRegistry(leClient, leAlias, 'leRegistry')

Using Passcode (bran): BqoaJc5uPDqKVnmAWPgBt
Client boot process initiated with KERIA agent.
  Client AID Prefix:  EAVZ2wSunSvjss_JPSbUIgZW6C48PrQ78zrwYh5V-fJd
  Agent AID Prefix:   EGBHjS_qPLWfHqv4n-FvhTZNzhYElWo0_Kb4nA1-bAIZ
Initiating AID inception for alias: gleif
Successfully created AID with prefix: EBXH9BX9G-ohOpzhlKfBzzAht-35g9DxRDgDwEnIy2Lt
Assigning 'agent' role to KERIA Agent EGBHjS_qPLWfHqv4n-FvhTZNzhYElWo0_Kb4nA1-bAIZ for AID alias gleif
Successfully assigned 'agent' role for AID alias gleif.
Generating OOBI for AID alias gleif with role agent
Generated OOBI URL: http://keria:3902/oobi/EBXH9BX9G-ohOpzhlKfBzzAht-35g9DxRDgDwEnIy2Lt/agent/EGBHjS_qPLWfHqv4n-FvhTZNzhYElWo0_Kb4nA1-bAIZ
Using Passcode (bran): D0zjYJMaX_waCgpU7jbOV
Client boot process initiated with KERIA agent.
  Client AID Prefix:  ED-DHukbbwy8gNde17QbRfLzPI53I9YvJFVrOKsXMZHU
  Agent AID Prefix:   EK9zEekpdAcmat9PcP9O3u4lko_zvL_3eX7CnGULGFQq
Initiating AID inception for alias: qvi
Successfully created AID with p

[
  {
    operation: {
      name: "oobi.0AA6DWzKEN2q3yJvSlykeO07",
      metadata: {
        oobi: "http://keria:3902/oobi/EGzk7s5KZaKvTdhsofGPfwlHq3WWcCTNB6cdCNTeceFL/agent/EK9zEekpdAcmat9PcP9O3u4lko_zvL_3eX7CnGULGFQq"
      },
      done: true,
      error: null,
      response: {
        vn: [ 1, 0 ],
        i: "EGzk7s5KZaKvTdhsofGPfwlHq3WWcCTNB6cdCNTeceFL",
        s: "0",
        p: "",
        d: "EGzk7s5KZaKvTdhsofGPfwlHq3WWcCTNB6cdCNTeceFL",
        f: "0",
        dt: "2025-06-12T21:58:55.049505+00:00",
        et: "icp",
        kt: "1",
        k: [ "DArwLqrtw3iJWA7InUsF4_We9hCn2YT38ScB8TQ_4nSw" ],
        nt: "1",
        n: [ "EJLr9y3GL2wv_PlKNUwLoicCY5uAWY1aizB8HhI-zHK0" ],
        bt: "3",
        b: [
          "BBilc4-L3tFUnfM_wJr4S4OJanAv_VmF_dJNN6vkf2Ha",
          "BLskRTInXnMxWaGqcpSyMgo0nYbalW99cGZESrz3zapM",
          "BIKKuvBwpmDVA4Ds-EpL5bt9OqPzWPja2LigFYZN2YfX"
        ],
        c: [],
        ee: {
          s: "0",
          d: "EGzk7s5KZaKvTdhsofGPfwlHq3

Resolves schema definitions for different credential types

Talk about the different schemas and list them:

- **[QVI Credential](config/schemas/qualified-vLEI-issuer-vLEI-credential.json)**
- **[vLEI Credential](config/schemas/legal-entity-vLEI-credential.json)**
- **[OOR Auth Credential](config/schemas/oor-authorization-vLEI-credential.json)**
- **[ECR Auth Credential](config/schemas/ecr-authorization-vlei-credential.json)**
- **[OOR Credential](config/schemas/legal-entity-official-organizational-role-vLEI-credential.json)**
- **[ECR Credential](config/schemas/legal-entity-engagement-context-role-vLEI-credential.json)**



note: Schemas are preloaded

In [2]:
// Schemas

// vLEI Schema SAIDs. These are well known schemas. Already preloaded
const QVI_SCHEMA_SAID = 'EBfdlu8R27Fbx-ehrqwImnK-8Cm79sqbAQ4MmvEAYqao';
const LE_SCHEMA_SAID = 'ENPXp1vQzRF6JwIuS-mp2U8Uf1MoADoP_GqQ62VsDZWY';
const ECR_AUTH_SCHEMA_SAID = 'EH6ekLjSr8V32WyFbGe1zXjTzFs9PkTYmupJ9H65O14g';
const ECR_SCHEMA_SAID = 'EEy9PkikFcANV1l7EHukCeXqrzT1hNZjGlUk7wuMO5jw';
const OOR_AUTH_SCHEMA_SAID = 'EKA57bKBKxr_kN7iN5i7lMUxpMG-s19dRcmov1iDxz-E';
const OOR_SCHEMA_SAID = 'EBNaNu-M9P5cgrnfl2Fvymy4E_jvxxyjb70PRtiANlJy';

const schemaServer = `http://vlei-server:7723/oobi`;
const QVI_SCHEMA_URL = `${schemaServer}/${QVI_SCHEMA_SAID}`;
const LE_SCHEMA_URL = `${schemaServer}/${LE_SCHEMA_SAID}`;
const ECR_AUTH_SCHEMA_URL = `${schemaServer}/${ECR_AUTH_SCHEMA_SAID}`;
const ECR_SCHEMA_URL = `${schemaServer}/${ECR_SCHEMA_SAID}`;
const OOR_AUTH_SCHEMA_URL = `${schemaServer}/${OOR_AUTH_SCHEMA_SAID}`;
const OOR_SCHEMA_URL = `${schemaServer}/${OOR_SCHEMA_SAID}`;

Resolving schemas

In [3]:
await Promise.all([
    resolveOOBI(gleifClient, QVI_SCHEMA_URL),
    
    resolveOOBI(qviClient, QVI_SCHEMA_URL),
    resolveOOBI(qviClient, LE_SCHEMA_URL),
    resolveOOBI(qviClient, ECR_AUTH_SCHEMA_URL),
    resolveOOBI(qviClient, ECR_SCHEMA_URL),
    resolveOOBI(qviClient, OOR_AUTH_SCHEMA_URL),
    resolveOOBI(qviClient, OOR_SCHEMA_URL),
    
    resolveOOBI(leClient, QVI_SCHEMA_URL),
    resolveOOBI(leClient, LE_SCHEMA_URL),
    resolveOOBI(leClient, ECR_AUTH_SCHEMA_URL),
    resolveOOBI(leClient, ECR_SCHEMA_URL),
    resolveOOBI(leClient, OOR_AUTH_SCHEMA_URL),
    resolveOOBI(leClient, OOR_SCHEMA_URL),
    
    resolveOOBI(roleClient, QVI_SCHEMA_URL),
    resolveOOBI(roleClient, LE_SCHEMA_URL),
    resolveOOBI(roleClient, ECR_AUTH_SCHEMA_URL),
    resolveOOBI(roleClient, ECR_SCHEMA_URL),
    resolveOOBI(roleClient, OOR_AUTH_SCHEMA_URL),
    resolveOOBI(roleClient, OOR_SCHEMA_URL),
]);

Resolving OOBI URL: http://vlei-server:7723/oobi/EBfdlu8R27Fbx-ehrqwImnK-8Cm79sqbAQ4MmvEAYqao with alias undefined
Resolving OOBI URL: http://vlei-server:7723/oobi/EBfdlu8R27Fbx-ehrqwImnK-8Cm79sqbAQ4MmvEAYqao with alias undefined
Resolving OOBI URL: http://vlei-server:7723/oobi/ENPXp1vQzRF6JwIuS-mp2U8Uf1MoADoP_GqQ62VsDZWY with alias undefined
Resolving OOBI URL: http://vlei-server:7723/oobi/EH6ekLjSr8V32WyFbGe1zXjTzFs9PkTYmupJ9H65O14g with alias undefined
Resolving OOBI URL: http://vlei-server:7723/oobi/EEy9PkikFcANV1l7EHukCeXqrzT1hNZjGlUk7wuMO5jw with alias undefined
Resolving OOBI URL: http://vlei-server:7723/oobi/EKA57bKBKxr_kN7iN5i7lMUxpMG-s19dRcmov1iDxz-E with alias undefined
Resolving OOBI URL: http://vlei-server:7723/oobi/EBNaNu-M9P5cgrnfl2Fvymy4E_jvxxyjb70PRtiANlJy with alias undefined
Resolving OOBI URL: http://vlei-server:7723/oobi/EBfdlu8R27Fbx-ehrqwImnK-8Cm79sqbAQ4MmvEAYqao with alias undefined
Resolving OOBI URL: http://vlei-server:7723/oobi/ENPXp1vQzRF6JwIuS-mp2U8Uf1MoADo

[
  {
    operation: {
      name: "oobi.0ACdX_4vgyiogMwPTQaQiAhJ",
      metadata: {
        oobi: "http://vlei-server:7723/oobi/EBfdlu8R27Fbx-ehrqwImnK-8Cm79sqbAQ4MmvEAYqao"
      },
      done: true,
      error: null,
      response: {
        oobi: "http://vlei-server:7723/oobi/EBfdlu8R27Fbx-ehrqwImnK-8Cm79sqbAQ4MmvEAYqao"
      }
    },
    contact: [
      {
        id: "BBilc4-L3tFUnfM_wJr4S4OJanAv_VmF_dJNN6vkf2Ha",
        alias: "wan",
        oobi: "http://witness-demo:5642/oobi/BBilc4-L3tFUnfM_wJr4S4OJanAv_VmF_dJNN6vkf2Ha/controller?name=wan&tag=witness&tag=sample",
        ends: { controller: [Object] },
        challenges: [],
        wellKnowns: []
      },
      {
        id: "BF2rZTW79z4IXocYRQnjjsOuvFUQv-ptCf8Yltd7PfsM",
        alias: "wyz",
        oobi: "http://witness-demo:5647/oobi/BF2rZTW79z4IXocYRQnjjsOuvFUQv-ptCf8Yltd7PfsM/controller?name=wyz&tag=witness&tag=sample",
        ends: { controller: [Object] },
        challenges: [],
        wellKnowns: []
      }

## Credential Issuance Chain
The test follows the official vLEI ecosystem hierarchy:

- QVI Credential: GLEIF issues a Qualified vLEI Issuer credential to the QVI
- LE Credential: QVI issues a Legal Entity credential to the LE
- ECR Credential (Path 1): LE directly issues an Engagement Context Role credential to the Role holder
- ECR AUTH Credential: LE issues an ECR authorization credential to the QVI
- ECR Credential (Path 2): QVI issues another ECR credential using the AUTH credential
- OOR AUTH Credential: LE issues an Official Organizational Role authorization to QVI
- OOR Credential: QVI issues the final OOR credential to the Role holder

### QVI Credential: GLEIF issues a Qualified vLEI Issuer credential to the QVI


In [4]:
// QVI LEI (Arbitrary value)
const qviData = {
    LEI: '254900OPPU84GM83MG36',
};

// GLEIF - Issue credential
const { credentialSaid: credentialSaid} = await issueCredential(
    gleifClient, 
    gleifAlias, 
    gleifRegistrySaid, 
    QVI_SCHEMA_SAID,
    qviAid.i,
    qviData
)

// GLEIF - get credential
const qviCredential = await gleifClient.credentials().get(credentialSaid);

// GLEIF - Ipex grant
const grantResponse = await ipexGrantCredential(
    gleifClient,
    gleifAlias, 
    qviAid.i,
    qviCredential
)

// QVI - Wait for grant notification
const grantNotifications = await waitForAndGetNotification(qviClient, IPEX_GRANT_ROUTE)
const grantNotification = grantNotifications[0]

// QVI - Admit Grant
const admitResponse = await ipexAdmitGrant(
    qviClient,
    qviAlias,
    gleifAid.i,
    grantNotification.a.d
)

// QVI - Mark notification
await markNotificationRead(qviClient, grantNotification.i)

// GLEIF - Wait for admit notification
const admitNotifications = await waitForAndGetNotification(gleifClient, IPEX_ADMIT_ROUTE)
const admitNotification = admitNotifications[0]

// GLEIF - Mark notification
await markNotificationRead(gleifClient, admitNotification.i)

Issuing credential from AID "gleif" to AID "EGzk7s5KZaKvTdhsofGPfwlHq3WWcCTNB6cdCNTeceFL"...
Successfully created credential registry: EHki4Cf6HC531_tSF0WIq6jnflA-KIYSjKfojXjdz0xr
Successfully created credential registry: EHNMTVq73qITp7Lel9xnrnSXTTO7sqbg0bnuVDGUC6A_
{
  name: "credential.EDxncvoJeDc9U_7D32kpdqggPrJJvkl7vZd-xmUJIOwL",
  metadata: {
    ced: {
      v: "ACDC10JSON000197_",
      d: "EDxncvoJeDc9U_7D32kpdqggPrJJvkl7vZd-xmUJIOwL",
      i: "EBXH9BX9G-ohOpzhlKfBzzAht-35g9DxRDgDwEnIy2Lt",
      ri: "EAkkDRthKw0cPvMvFbMI_hpRiqL2Qp5kBMGsRvyT_dav",
      s: "EBfdlu8R27Fbx-ehrqwImnK-8Cm79sqbAQ4MmvEAYqao",
      a: {
        d: "EMtgXcPNp8ymdrqWxsx54f1J5uOvI7jFYmAGEVMwy4T1",
        i: "EGzk7s5KZaKvTdhsofGPfwlHq3WWcCTNB6cdCNTeceFL",
        LEI: "254900OPPU84GM83MG36",
        dt: "2025-06-12T21:58:58.015000+00:00"
      }
    },
    depends: {
      name: "witness.EDEQ-rwst4cXjwX7UP0LCqWny1Dt_C8da9FBNEnA6tBf",
      metadata: { pre: "EBXH9BX9G-ohOpzhlKfBzzAht-35g9DxRDgDwEnIy2Lt"

### LE Credential: QVI issues a Legal Entity credential to the LE


In [ ]:
// qvi - Issue credential
const { credentialSaid: credentialSaid} = await issueCredential(
    qviClient, 
    qviAlias, 
    qviRegistrySaid, 
    LE_SCHEMA_SAID,
    LEAid.i,
    LEData
)

// qvi - get credential (with all its data)
const credential = await qviClient.credentials().get(credentialSaid);

// qvi - Ipex grant
const grantResponse = await ipexGrantCredential(
    qviClient,
    qviAlias, 
    LEAid.i,
    credential
)

// LE - Wait for grant notification
const grantNotifications = await waitForAndGetNotification(LEClient, IPEX_GRANT_ROUTE)
const grantNotification = grantNotifications[0]

// LE - Admit Grant
const admitResponse = await ipexAdmitGrant(
    LEClient,
    LEAlias,
    qviAid.i,
    grantNotification.a.d
)

// LE - Mark notification
await markNotificationRead(LEClient, grantNotification.i)

// qvi - Wait for admit notification
const admitNotifications = await waitForAndGetNotification(qviClient, IPEX_ADMIT_ROUTE)
const admitNotification = admitNotifications[0]

// qvi - Mark notification
await markNotificationRead(qviClient, admitNotification.i)

### ECR Credential (Path 1): LE directly issues an Engagement Context Role credential to the Role holder


### ECR AUTH Credential: LE issues an ECR authorization credential to the QVI


### ECR Credential (Path 2): QVI issues another ECR credential using the AUTH credential


### OOR AUTH Credential: LE issues an Official Organizational Role authorization to QVI


### OOR Credential: QVI issues the final OOR credential to the Role holder

Summary:

Hierarchical trust: Each credential references its authorizing credential
Multiple issuance paths: Shows both direct issuance (LE→Role) and authorized issuance (LE→QVI→Role)
IPEX protocol: Uses grant/admit message exchange for credential delivery
Schema compliance: Each credential follows specific vLEI schemas
Credential chaining: Later credentials reference earlier ones as sources of authority
This represents a simplified vLEI ecosystem where organizations can issue verifiable credentials about legal entities and their roles while maintaining a proper chain of trust.